# Part 1: Data Acquisition, Multi-Year Consolidation, and Initial Processing

**Objective:** Create a clean, consolidated, multi-year dataset from raw annual trade data. This involves:
1.  **Loading and Merging:** Combining individual yearly CSV files (1988-2024) from the `raw_import_data/` folder into a single DataFrame. The initial merge resulted in 795,683 records.
2.  **Data Cleaning and Schema Standardization:** Extracting the 'Year' from the `refPeriodId` column, renaming key columns to a consistent schema (`importer`, `exporter`, `amount`, `year`), and filtering out non-country 'World' entries.
3.  **Geographic Feature Enrichment:** Augmenting the data by adding latitude and longitude coordinates for both importer and exporter countries from a lookup file (`country-coord.csv`). Records with missing coordinates were dropped.
4.  **Significance Filtering:** Applying a "Top 20" filter to retain only the most significant trade partners for each importer on an annual basis, focusing the analysis on the most impactful trade relationships.
5.  **Final Output Generation:** Saving the final processed and filtered dataset, which contains 17,170 observations, to `trade_data.csv`.
6.  **Workspace Cleanup:** Removing all intermediate temporary files generated during the process.

## 1. Configuration and Setup
Import necessary libraries and define configuration variables for file paths, folder locations, and processing parameters like 'Top N'.

In [1]:
import pandas as pd
import glob
import os
import re # For robust year extraction from filenames

# --- Configuration ---
RAW_DATA_FOLDER = 'raw_import_data/' # Folder containing yearly CSV files (e.g., 2023.csv)
COORDS_FILE = 'country-coord.csv'    # File with country coordinates
TOP_N_PARTNERS = 20                  # Number of top partners to keep for each importer per year

# Define temporary and final output filenames
TEMP_MERGED_CLEANED_FILE = 'temp_merged_cleaned_data.csv'
TEMP_WITH_COORDS_FILE = 'temp_data_with_coords.csv'
FINAL_OUTPUT_FILE_PART1 = 'trade_data.csv'

## 2. Load and Merge Yearly Data Files
This step iterates through all `.csv` files located in the `RAW_DATA_FOLDER`. For each file, the year is parsed from the filename (e.g., '1988.csv' -> 1988). The data from each year is loaded into a pandas DataFrame and then concatenated into a single `master_df`. The output confirms that data from 1988 to 2024 was successfully loaded, resulting in a master DataFrame with 795,683 rows and 48 columns before cleaning.

In [2]:
all_files = glob.glob(os.path.join(RAW_DATA_FOLDER, "*.csv"))
list_of_dfs = []

if not all_files:
    print(f"Warning: No CSV files found in {RAW_DATA_FOLDER}. Please check the path.")

for f in all_files:
    try:
        filename = os.path.basename(f)
        # Attempt to extract a 4-digit year from the filename robustly
        year_str = filename.split('.')[0] # Initial attempt
        if not year_str.isdigit() or len(year_str) != 4:
            match = re.search(r'(\d{4})', filename) # Regex fallback
            if match:
                year_str = match.group(1)
            else:
                # If year cannot be determined, it might be an issue or the column 'refPeriodId' is relied upon solely
                raise ValueError(f"Could not extract 4-digit year from filename: {filename}")
        year_from_filename = int(year_str)
        
        df_temp = pd.read_csv(f, encoding="latin1", low_memory=False)
        df_temp['Year'] = year_from_filename # This 'Year' is from filename
        list_of_dfs.append(df_temp)
        print(f"Processed: {filename} for year {year_from_filename}")
    except Exception as e:
        print(f"Error processing file {f}: {e}")

if not list_of_dfs:
    raise FileNotFoundError(f"No CSV files found or successfully processed in {RAW_DATA_FOLDER}. Project cannot continue.")

master_df = pd.concat(list_of_dfs, ignore_index=True)

print(f"\nMaster DataFrame shape after merging: {master_df.shape}")
print("Master DataFrame Info:")
master_df.info()

Processed: 1988.csv for year 1988
Processed: 1989.csv for year 1989
Processed: 1990.csv for year 1990
Processed: 1991.csv for year 1991
Processed: 1992.csv for year 1992
Processed: 1993.csv for year 1993
Processed: 1994.csv for year 1994
Processed: 1995.csv for year 1995
Processed: 1996.csv for year 1996
Processed: 1997.csv for year 1997
Processed: 1998.csv for year 1998
Processed: 1999.csv for year 1999
Processed: 2000.csv for year 2000
Processed: 2001.csv for year 2001
Processed: 2002.csv for year 2002
Processed: 2003.csv for year 2003
Processed: 2004.csv for year 2004
Processed: 2005.csv for year 2005
Processed: 2006.csv for year 2006
Processed: 2007.csv for year 2007
Processed: 2008.csv for year 2008
Processed: 2009.csv for year 2009
Processed: 2010.csv for year 2010
Processed: 2011.csv for year 2011
Processed: 2012.csv for year 2012
Processed: 2013.csv for year 2013
Processed: 2014.csv for year 2014
Processed: 2015.csv for year 2015
Processed: 2016.csv for year 2016
Processed: 201

## 3. Initial Cleaning, Renaming, and Filtering
The `master_df` is subsetted to retain only the essential columns for the analysis: `reporterISO`, `partnerISO`, `cifvalue`, and `refPeriodId`. These columns are then renamed to a more intuitive and standardized schema: `importer`, `exporter`, `amount`, and `year`, respectively. The `year` column is derived from the `refPeriodId` in the raw data, making it the authoritative source for the time period. Any rows where the importer or exporter is listed as 'World' are removed, and records with non-numeric or missing trade `amount` values are dropped. The resulting cleaned DataFrame is then saved to a temporary file for the next stage.

In [3]:
# Ensure required columns for renaming exist in master_df
required_rename_cols = {'reporterISO', 'partnerISO', 'cifvalue', 'refPeriodId'}
if not required_rename_cols.issubset(master_df.columns):
    missing = required_rename_cols - set(master_df.columns)
    raise KeyError(f"One or more required columns for renaming are missing from master_df: {missing}. Available columns: {master_df.columns.tolist()}")

df_cleaned = master_df[['reporterISO', 'partnerISO', 'cifvalue', 'refPeriodId']].rename(
    columns={
        'reporterISO': 'importer',
        'partnerISO': 'exporter',
        'cifvalue': 'amount',
        'refPeriodId': 'year' # 'refPeriodId' from data is now the primary 'year'
    }
).copy() # Use .copy() to ensure it's a new DataFrame

# Convert 'amount' to numeric, errors will be coerced to NaN
df_cleaned['amount'] = pd.to_numeric(df_cleaned['amount'], errors='coerce')
# Remove rows where 'amount' became NaN after conversion or was originally NaN
df_cleaned.dropna(subset=['amount'], inplace=True)

df_cleaned = df_cleaned[
    (df_cleaned['importer'] != 'World') &
    (df_cleaned['exporter'] != 'World')
]

print(f"\nCleaned DataFrame head (before saving to temp file):")
print(df_cleaned.head())
df_cleaned.to_csv(TEMP_MERGED_CLEANED_FILE, index=False)


Cleaned DataFrame head (before saving to temp file):
    importer     exporter       amount  year
1  Australia  Afghanistan     237515.0  1988
2  Australia      Albania       4536.0  1988
3  Australia      Algeria      96230.0  1988
4  Australia    Argentina   41170679.0  1988
5  Australia      Austria  114447116.0  1988


## 4. Adding Geographic Coordinates
The cleaned trade data is enriched with geographic information. A lookup file, `country-coord.csv`, is used to map latitude and longitude to both the importer and exporter of each trade record based on their ISO codes or country names. This step is crucial for later visualization and for calculating geographic distance if needed. Any trade records where coordinates could not be found for either the importer or the exporter are dropped from the dataset to ensure data integrity. The augmented DataFrame is then saved to a new temporary file.

In [4]:
df_for_coords = pd.read_csv(TEMP_MERGED_CLEANED_FILE)
df_country_coords_lookup = pd.read_csv(COORDS_FILE)

alpha3_coords = df_country_coords_lookup.set_index('Alpha-3 code')[['Latitude (average)', 'Longitude (average)']].to_dict(orient='index')
country_coords = df_country_coords_lookup.set_index('Country')[['Latitude (average)', 'Longitude (average)']].to_dict(orient='index')

def get_coordinates(value):
    if value in alpha3_coords:
        return (alpha3_coords[value]['Latitude (average)'], 
                alpha3_coords[value]['Longitude (average)'])
    elif value in country_coords:
        return (country_coords[value]['Latitude (average)'], 
                country_coords[value]['Longitude (average)'])
    else:
        return (None, None)

df_for_coords['importer_latitude'], df_for_coords['importer_longitude'] = zip(*df_for_coords['importer'].apply(get_coordinates))
df_for_coords['exporter_latitude'], df_for_coords['exporter_longitude'] = zip(*df_for_coords['exporter'].apply(get_coordinates))

df_for_coords.dropna(subset=['importer_latitude', 'importer_longitude', 
                               'exporter_latitude', 'exporter_longitude'], inplace=True)

df_for_coords.to_csv(TEMP_WITH_COORDS_FILE, index=False)

## 5. "Top N" Partner Filtering and Final Save
To focus the analysis on the most economically significant relationships and maintain computational feasibility, the dataset is filtered. The data is sorted by `year`, `importer`, and trade `amount` (descending). Then, for each importer within each year, only the top 20 trade partners (as defined by `TOP_N_PARTNERS`) are retained. This filtering step reduces the dataset to 17,170 core trade flows. The final, processed DataFrame for Part 1 is then saved to `trade_data.csv` to be used in subsequent analysis parts.

In [5]:
df_for_topn_filter = pd.read_csv(TEMP_WITH_COORDS_FILE)
df_for_topn_filter = df_for_topn_filter.sort_values(by=['year', 'importer', 'amount'], ascending=[True, True, False])
df_final_part1 = df_for_topn_filter.groupby(['year', 'importer']).head(TOP_N_PARTNERS).reset_index(drop=True)

print(f"\nFinal Top N DataFrame shape: {df_final_part1.shape}")
print("Final Top N DataFrame Info:")
df_final_part1.info()
print("\nFinal Top N DataFrame Head:")
print(df_final_part1.head())

df_final_part1.to_csv(FINAL_OUTPUT_FILE_PART1, index=False)
print(f"\nFinal processed data for Part 1 saved to: {FINAL_OUTPUT_FILE_PART1}")


Final Top N DataFrame shape: (17170, 8)
Final Top N DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17170 entries, 0 to 17169
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   importer            17170 non-null  object 
 1   exporter            17170 non-null  object 
 2   amount              17170 non-null  float64
 3   year                17170 non-null  int64  
 4   importer_latitude   17170 non-null  float64
 5   importer_longitude  17170 non-null  float64
 6   exporter_latitude   17170 non-null  float64
 7   exporter_longitude  17170 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 1.0+ MB

Final Top N DataFrame Head:
    importer        exporter        amount  year  importer_latitude  \
0  Australia             USA  7.041979e+09  1988              -27.0   
1  Australia           Japan  6.549632e+09  1988              -27.0   
2  Australia  United Kingdom 

## 6. Cleanup Temporary Files
To maintain a clean project directory, the intermediate files (`temp_merged_cleaned_data.csv` and `temp_data_with_coords.csv`) that were created during the preprocessing pipeline are programmatically removed.

In [6]:
temp_files_to_remove = [TEMP_MERGED_CLEANED_FILE, TEMP_WITH_COORDS_FILE]
for temp_file in temp_files_to_remove:
    try:
        if os.path.exists(temp_file):
            os.remove(temp_file)
            print(f"Successfully removed temporary file: {temp_file}")
        else:
            print(f"Temporary file not found (already removed or never created): {temp_file}")
    except Exception as e:
        print(f"Error removing temporary file {temp_file}: {e}")

print("\nPart 1 processing complete.")

Successfully removed temporary file: temp_merged_cleaned_data.csv
Successfully removed temporary file: temp_data_with_coords.csv

Part 1 processing complete.
